In [170]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import ultraplot as plot 
import seaborn as sns
import colorcet as cc

import uproot
import mplhep as hep
import hist
import hist.intervals
import hist.plot

In [171]:

data = uproot.open('CCNp_visualScan.root:scanning')
data.keys()

['POT;5',
 'POT;4',
 'POT;3',
 'POT;2',
 'POT;1',
 'Livetime;5',
 'Livetime;4',
 'Livetime;3',
 'Livetime;2',
 'Livetime;1',
 'true_nominal;1',
 'true_cheated_2d;1',
 'true_cheated_2d_vtx;1',
 'true_cheated_2d_vtx_3d;1',
 'true_cheated_2d_vtx_3d_mva;1']

In [172]:
class Component:
    def __init__(self, data, tree_base='{}'):
        self.data = data
        self.tree_base = tree_base

    def get(self, 
            stage1: str | list[str], 
            product: str | list[str], 
            binning, 
            product_thr: float | list[float] = -100,
            conditions: dict | None = None
           ):

        if not isinstance(stage1, list):
            stage1 = [stage1]

        if not isinstance(product, list):
            product = [product]

        if not isinstance(product_thr, list):
            product_thr = [product_thr] * len(product)
        

        if not isinstance(binning, list) and len(product) > 1:
            binning = [binning] * len(product)
        
        data = []
        hists = []
    
        for e in stage1:
            data.append(self.data[self.tree_base.format(e)].arrays(library='pd'))

        
        d0 = data[0]
        common = d0
        for i in range(1, len(data)):
            d = data[i]
            common = pd.merge(
                d0[(d0[product] > product_thr).all(axis=1)][['Evt', *product]], 
                d[(d[product] > product_thr).all(axis=1)][['Evt', *product]], 
                on='Evt', suffixes=('', '_more')
            )
            # print(common.keys())
            d0 = common

        return_data = []
        for d in data:
            if len(product) == 1:
                if conditions is not None:
                    for k in conditions:
                        d = d[d[k] == conditions[k]]
                # hists.append(hist.Hist(binning).fill(d[d.Evt.isin(common.Evt)][product[0]].fillna(0).values))
            else:
                fill_data = [d[d.Evt.isin(common.Evt)][p].values for p in product]
                # hists.append(hist.Hist(*binning).fill(*fill_data))
                return_data.append(d[d.Evt.isin(common.Evt)][['Evt', *product]])
            if False:
                print('ERROR: only 1D and 2D histograms are supported. The provided list of products exceed dimensionality: {}'.format(product))
            
        return return_data
            
    def keys(self, stage1):
        return self.data[self.tree_base.format(stage1)].keys()

In [173]:

visualScanningAna = Component(data, tree_base='true_{}')


In [174]:
visualScanningAna.keys('nominal')

['muonLength',
 'protonMinLength',
 'vertexX',
 'vertexY',
 'vertexZ',
 'selected',
 'Run',
 'Subrun',
 'Evt']

In [175]:
nominal = data['true_nominal'].arrays(library='pd')
cheated_2d = data['true_cheated_2d'].arrays(library='pd')
cheated_2d_vtx = data['true_cheated_2d_vtx'].arrays(library='pd')
cheated_2d_vtx_3d = data['true_cheated_2d_vtx_3d'].arrays(library='pd')
cheated_2d_vtx_3d_mva = data['true_cheated_2d_vtx_3d_mva'].arrays(library='pd')


nominal['EvtRun'] = nominal.Evt*10000 + nominal.Run
cheated_2d['EvtRun'] = cheated_2d.Evt*10000 + cheated_2d.Run
cheated_2d_vtx['EvtRun'] = cheated_2d_vtx.Evt*10000 + cheated_2d_vtx.Run
cheated_2d_vtx_3d['EvtRun'] = cheated_2d_vtx_3d.Evt*10000 + cheated_2d_vtx_3d.Run
cheated_2d_vtx_3d_mva['EvtRun'] = cheated_2d_vtx_3d_mva.Evt*10000 + cheated_2d_vtx_3d_mva.Run

cryoRequest = (
    (nominal['vertexX'] > 0) & 
    (cheated_2d['vertexX'] > 0) & 
    (cheated_2d_vtx['vertexX'] > 0) & 
    (cheated_2d_vtx_3d['vertexX'] > 0) & 
    (cheated_2d_vtx_3d_mva['vertexX'] > 0)
)

nominal = nominal[cryoRequest]
cheated_2d = cheated_2d[cryoRequest]
cheated_2d_vtx = cheated_2d_vtx[cryoRequest]
cheated_2d_vtx_3d = cheated_2d_vtx_3d[cryoRequest]
cheated_2d_vtx_3d_mva = cheated_2d_vtx_3d_mva[cryoRequest]

/var/folders/rz/xbjc5djs43l2yfd7lq67www80000gn/T/ipykernel_4615/291027018.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cheated_2d = cheated_2d[cryoRequest]
/var/folders/rz/xbjc5djs43l2yfd7lq67www80000gn/T/ipykernel_4615/291027018.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cheated_2d_vtx = cheated_2d_vtx[cryoRequest]
/var/folders/rz/xbjc5djs43l2yfd7lq67www80000gn/T/ipykernel_4615/291027018.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cheated_2d_vtx_3d = cheated_2d_vtx_3d[cryoRequest]
/var/folders/rz/xbjc5djs43l2yfd7lq67www80000gn/T/ipykernel_4615/291027018.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cheated_2d_vtx_3d_mva = cheated_2d_vtx_3d_mva[cryoRequest]


In [176]:


com = pd.merge(
    nominal[['Run', 'Evt', 'vertexX', 'vertexY', 'vertexZ', 'selected', 'EvtRun']], 
    cheated_2d[['selected', 'EvtRun']], on='EvtRun', suffixes=('', '_cheated_2d')
)

com = pd.merge(com, cheated_2d_vtx[['selected', 'EvtRun']], on='EvtRun', suffixes=('', '_cheated_2d_vtx'))
com = pd.merge(com, cheated_2d_vtx_3d[['selected', 'EvtRun']], on='EvtRun', suffixes=('', '_cheated_2d_vtx_3d'))
com = pd.merge(com, cheated_2d_vtx_3d_mva[['selected', 'EvtRun']], on='EvtRun', suffixes=('', '_cheated_2d_vtx_3d_mva'))


            

In [177]:
maskSel = (
    (com.selected > -9000) &
    (com.selected_cheated_2d > -9000) &
    (com.selected_cheated_2d_vtx > -9000) &
    (com.selected_cheated_2d_vtx_3d > -9000) &
    (com.selected_cheated_2d_vtx_3d_mva > -9000)
)

save = com[maskSel].head(300)
save = save.drop('EvtRun', axis=1)
save.to_csv('selectedForChristian.csv', index=False)

In [178]:
com[maskSel]

,Run,Evt,vertexX,vertexY,vertexZ,selected,EvtRun,selected_cheated_2d,selected_cheated_2d_vtx,selected_cheated_2d_vtx_3d,selected_cheated_2d_vtx_3d_mva
0,1,17,213.230362,-47.447048,-308.043549,1.0,170001,0.0,0.0,0.0,0.0
1,1000,18,184.840500,-2.090725,304.816010,0.0,181000,0.0,0.0,0.0,0.0
2,1001,21,196.257370,50.764591,52.596020,1.0,211001,1.0,1.0,1.0,1.0
3,1003,5,123.013321,-67.164215,51.088886,1.0,51003,1.0,1.0,1.0,1.0
4,1003,24,113.310806,-128.698990,194.813370,0.0,241003,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1943,2958,8,230.700226,-38.722187,-471.109009,0.0,82958,0.0,0.0,0.0,0.0
1944,2960,37,254.959030,-28.412689,498.533325,1.0,372960,1.0,1.0,1.0,1.0
1945,2961,2,303.636078,-33.574932,-760.753601,0.0,22961,0.0,0.0,0.0,0.0
1946,2963,34,265.994263,-122.091797,-439.533722,0.0,342963,0.0,0.0,0.0,0.0
